<a href="https://colab.research.google.com/github/lkhok22/ML-hw4/blob/main/face-recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
! pip install kaggle

In [7]:
import wandb
wandb.login()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lkhok22 (lkhok22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


 90% 257M/285M [00:00<00:00, 317MB/s]
100% 285M/285M [00:00<00:00, 349MB/s]


In [10]:
import pandas as pd
import numpy as np

In [14]:
train_df_t = pd.read_csv('train.csv')
test_df_t = pd.read_csv('test.csv')

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import random
import os

# Set seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()


In [17]:
from PIL import Image

class FERDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = np.array(row['pixels'].split(), dtype=np.uint8).reshape(48, 48)
        label = int(row['emotion']) if 'emotion' in row else -1
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image, label


In [18]:
from sklearn.model_selection import train_test_split

# Split training data for validation
train_df, val_df = train_test_split(train_df_t, test_size=0.1, stratify=train_df_t['emotion'], random_state=42)

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = FERDataset(train_df, transform=transform)
val_dataset = FERDataset(val_df, transform=transform)

# Use very small subset (e.g. 20 images) to test overfitting
small_train_dataset = torch.utils.data.Subset(train_dataset, list(range(20)))

small_loader = DataLoader(small_train_dataset, batch_size=4, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [19]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 24 * 24, 7)  # 7 classes
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x


In [20]:
def train_model(model, loader, optimizer, criterion, epochs=10, use_wandb=False):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, Accuracy: {acc:.4f}")
        if use_wandb:
            wandb.log({"loss": total_loss, "accuracy": acc})


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wandb.init(project="fer-overfit-test", name="SimpleCNN-overfit-small")

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, small_loader, optimizer, criterion, epochs=20, use_wandb=True)

wandb.finish()


Epoch 1/20, Loss: 11.3290, Accuracy: 0.2500
Epoch 2/20, Loss: 5.9664, Accuracy: 0.5500
Epoch 3/20, Loss: 3.2814, Accuracy: 0.9000
Epoch 4/20, Loss: 2.0939, Accuracy: 0.9500
Epoch 5/20, Loss: 1.4763, Accuracy: 1.0000
Epoch 6/20, Loss: 0.9487, Accuracy: 1.0000
Epoch 7/20, Loss: 0.6548, Accuracy: 1.0000
Epoch 8/20, Loss: 0.4465, Accuracy: 1.0000
Epoch 9/20, Loss: 0.3149, Accuracy: 1.0000
Epoch 10/20, Loss: 0.2425, Accuracy: 1.0000
Epoch 11/20, Loss: 0.1923, Accuracy: 1.0000
Epoch 12/20, Loss: 0.1482, Accuracy: 1.0000
Epoch 13/20, Loss: 0.1243, Accuracy: 1.0000
Epoch 14/20, Loss: 0.1016, Accuracy: 1.0000
Epoch 15/20, Loss: 0.0923, Accuracy: 1.0000
Epoch 16/20, Loss: 0.0796, Accuracy: 1.0000
Epoch 17/20, Loss: 0.0677, Accuracy: 1.0000
Epoch 18/20, Loss: 0.0600, Accuracy: 1.0000
Epoch 19/20, Loss: 0.0542, Accuracy: 1.0000
Epoch 20/20, Loss: 0.0493, Accuracy: 1.0000


accuracy,▁▄▇█████████████████
loss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1
loss,0.04933


In [22]:
wandb.init(project="fer-full-training", name="SimpleCNN-full")

model = SimpleCNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, optimizer, criterion, epochs=10, use_wandb=True)

wandb.finish()


Epoch 1/10, Loss: 674.7874, Accuracy: 0.3455
Epoch 2/10, Loss: 605.0413, Accuracy: 0.4306
Epoch 3/10, Loss: 568.9735, Accuracy: 0.4721
Epoch 4/10, Loss: 541.4911, Accuracy: 0.4989
Epoch 5/10, Loss: 521.6570, Accuracy: 0.5204
Epoch 6/10, Loss: 502.8986, Accuracy: 0.5404
Epoch 7/10, Loss: 488.7836, Accuracy: 0.5550
Epoch 8/10, Loss: 476.1859, Accuracy: 0.5663
Epoch 9/10, Loss: 464.2419, Accuracy: 0.5769
Epoch 10/10, Loss: 452.7581, Accuracy: 0.5925


accuracy,▁▃▅▅▆▇▇▇██
loss,█▆▅▄▃▃▂▂▁▁
accuracy,0.59246
loss,452.7581


In [23]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {correct / total:.4f}")


In [24]:
evaluate(model, val_loader)

Validation Accuracy: 0.4608
